## 1 Functions and module

### 1.1 Modules

In [2]:
import pandas as pd
import numpy as np
import copy
import scipy

### 1.2 Functions

In [3]:
def fdr(p_vals):
    from scipy.stats import rankdata
    p = np.asfarray(p_vals) # make input as float array
    by_descend = p.argsort()[::-1]
    by_orig = by_descend.argsort()
    p = p[by_descend] # sort pvalue from small to large
    ranked_p_values = rankdata(p,method ='max') # this max is very important, when identical, use largest
    fdr = p * len(p) / ranked_p_values
    fdr = np.minimum(1, np.minimum.accumulate(fdr))

    return fdr[by_orig]

## 2 Input and Output address 

### 2.1 Input

In [15]:
# combined barcode dataframe address
parent_address = "Data/"
Chromatin_bootstrap_address = parent_address + "Chromatin/Chromatin_bootstrapping_result_summary.csv"
Chromatin_bootstrap_KT_address = parent_address + "Chromatin/Chromatin_bootstrapping_result_KT_summary.csv"
# KT data
Saturation_bootstrap_address = parent_address + "Saturation/Saturation_bootstrapping_result_summary.csv"
Saturation_bootstrap_KT_address = parent_address + "Saturation/Saturation_bootstrapping_result_KT_summary.csv"

# sgRNA to include
Gene_to_include_address = parent_address + "Table_gene_group.xlsx"

### 2.2 Output 

In [79]:
LN_mean_output_address  = parent_address + 'Chromatin/Summary_table_LN_mean.csv'
Percentile_output_address  = parent_address + 'Chromatin/Summary_table_Percentile.csv'
TTN_output_address  = parent_address + 'Chromatin/Summary_table_TTN.csv'
TTB_output_address  = parent_address + 'Chromatin/Summary_table_TTB.csv'

## 3 Read data

In [6]:
def Generate_simplified(input_df):
    temp_q = [50,60,70,80,90,95,97,99]
    temp_trait_list = ['LN_mean_relative','Geo_mean_relative','TTB_normalized_relative','TTN_normalized_relative','95_percentile_relative'] + [str(x) + '_percentile_relative' for x in temp_q]
    temp_trait_list = list(set(temp_trait_list))
    temp_list = []
    for temp_trait in temp_trait_list:
        temp0 = temp_trait + '_97.5P'
        temp1 = temp_trait + '_2.5P'
        temp2 = temp_trait +'_bootstrap_mean'
        temp3 = temp_trait +'_pvalue'
        temp4 = temp_trait +'_pvalue_FDR'
        temp5 = temp_trait +'_bootstrap_median'
        temp_list.append(temp_trait)
        temp_list.append(temp0)
        temp_list.append(temp1)
        temp_list.append(temp2)
        temp_list.append(temp3)
        temp_list.append(temp4)
        temp_list.append(temp5)
    return input_df[['Targeted_gene_name','Numbered_gene_name','gRNA','Type'] +sorted(temp_list)]

In [8]:
# chromatin data
Chromatin_df = Generate_simplified(pd.read_csv(Chromatin_bootstrap_address))
Chromatin_KT_df = Generate_simplified(pd.read_csv(Chromatin_bootstrap_KT_address))

In [9]:
# chromatin data
Saturation_df = Generate_simplified(pd.read_csv(Saturation_bootstrap_address))
Saturation_KT_df = Generate_simplified(pd.read_csv(Saturation_bootstrap_KT_address))

### 3.1 I correct pvalues

In [10]:
temp_df = Chromatin_df
temp_c = [x for x in temp_df.columns if ('pvalue' in x)&( 'FDR' not in x)]
for x in temp_c:
    temp_df[x] = temp_df[x].apply(lambda x: min(x,1-x))
Chromatin_df = temp_df

In [11]:
temp_df = Chromatin_KT_df
temp_c = [x for x in temp_df.columns if ('pvalue' in x)&( 'FDR' not in x)]
for x in temp_c:
    temp_df[x] = temp_df[x].apply(lambda x: min(x,1-x))
Chromatin_KT_df = temp_df

In [12]:
temp_df = Saturation_df
temp_c = [x for x in temp_df.columns if ('pvalue' in x)&( 'FDR' not in x)]
for x in temp_c:
    temp_df[x] = temp_df[x].apply(lambda x: min(x,1-x))
Saturation_df = temp_df

In [13]:
temp_df = Saturation_KT_df
temp_c = [x for x in temp_df.columns if ('pvalue' in x)&( 'FDR' not in x)]
for x in temp_c:
    temp_df[x] = temp_df[x].apply(lambda x: min(x,1-x))
Saturation_KT_df = temp_df

### 3.2 Filter out genes from other people's data

In [44]:
# ref: https://docs.google.com/spreadsheets/d/1ucg3jBUXZMwZIv3QZhJ41Ne0RTrujqQ3/edit#gid=1593778826

In [45]:
Gene_to_include_df = pd.read_excel(Gene_to_include_address)

In [46]:
Gene_to_exclude_raw = ['CELSR1','CELSR2','VANGL1','Ptpn14','Cdkn1a','Gss',
                   'IFT80','CEP290','KIF3A','TULP3','IFT140','Stag1','Smc3','Smc1a',
                  'Nipbl','Mau2','Wapl','Pds5a','Rad21','OTUD5','UBE2Q1','MSI2','C7orf26',
                      'ZZZ3','RSU1','GCLC','WNK1','Sty1','Zmat3','Stag2']
#Sty1 is a typo version of gene Sytl1

In [47]:
Gene_to_exclude = [x.capitalize() for x in Gene_to_exclude_raw]

In [48]:
Chromatin_df_final = Chromatin_df[~Chromatin_df.Targeted_gene_name.isin(Gene_to_exclude)].copy()

In [49]:
Chromatin_KT_df_final = Chromatin_KT_df[~Chromatin_KT_df.Targeted_gene_name.isin(Gene_to_exclude)].copy()

In [50]:
Saturation_df_final = Saturation_df[~Saturation_df.Targeted_gene_name.isin(Gene_to_exclude)].copy()

In [51]:
Saturation_KT_df_final = Saturation_KT_df[~Saturation_KT_df.Targeted_gene_name.isin(Gene_to_exclude)].copy()

In [52]:
# test if all the remaining genes are expected 
kk = [x.capitalize() for x in Gene_to_include_df.Gene_symbol]
set([x.capitalize() for x in Chromatin_df_final.Targeted_gene_name]) - set(kk)

set()

### 3.3 Recalculate two side pvalue and FDR

In [53]:
temp_df = Chromatin_df_final
temp_c = [x for x in temp_df.columns if ('pvalue' in x)&( 'FDR' not in x)]
for x in temp_c:
    temp_df[x +'_FDR'] = fdr(temp_df[x])
    temp_df[x +'_twoside'] = temp_df[x]*2
    temp_df[x +'_twoside_FDR'] = fdr(temp_df[x +'_twoside'])
Chromatin_df_final = temp_df

In [54]:
temp_df = Chromatin_KT_df_final
temp_c = [x for x in temp_df.columns if ('pvalue' in x)&( 'FDR' not in x)]
for x in temp_c:
    temp_df[x +'_FDR'] = fdr(temp_df[x])
    temp_df[x +'_twoside'] = temp_df[x]*2
    temp_df[x +'_twoside_FDR'] = fdr(temp_df[x +'_twoside'])
Chromatin_KT_df_final = temp_df

In [55]:
temp_df = Saturation_df_final
temp_c = [x for x in temp_df.columns if ('pvalue' in x)&( 'FDR' not in x)]
for x in temp_c:
    temp_df[x +'_FDR'] = fdr(temp_df[x])
    temp_df[x +'_twoside'] = temp_df[x]*2
    temp_df[x +'_twoside_FDR'] = fdr(temp_df[x +'_twoside'])
Saturation_df_final = temp_df

In [56]:
temp_df = Saturation_KT_df_final
temp_c = [x for x in temp_df.columns if ('pvalue' in x)&( 'FDR' not in x)]
for x in temp_c:
    temp_df[x +'_FDR'] = fdr(temp_df[x])
    temp_df[x +'_twoside'] = temp_df[x]*2
    temp_df[x +'_twoside_FDR'] = fdr(temp_df[x +'_twoside'])
Saturation_KT_df_final = temp_df

### 3.4 Output_data

In [59]:
temp_a = parent_address + "Chromatin/Chromatin_sgRNA_result_final.csv"

In [60]:
Chromatin_df_final.to_csv(temp_a,index = False)

In [61]:
temp_a = parent_address + "Chromatin/Chromatin_sgRNA_result_KT_final.csv"

In [62]:
Chromatin_KT_df_final.to_csv(temp_a,index = False)

In [63]:
temp_a = parent_address + "Saturation/Saturation_sgRNA_result_final.csv"

In [64]:
Saturation_df_final.to_csv(temp_a,index = False)

In [65]:
temp_a = parent_address + "Saturation/Saturation_sgRNA_result_KT_final.csv"

In [66]:
Saturation_KT_df_final.to_csv(temp_a,index = False)

## 4 Generate metric specific table

In [67]:
def Generate_LN_mean_table(input_df):
    # generate a LN mean specific data table
    temp_trait = ['gRNA','Targeted_gene_name','Numbered_gene_name','Type','LN_mean_relative',
       'LN_mean_relative_2.5P', 'LN_mean_relative_bootstrap_mean','LN_mean_relative_bootstrap_median', 'LN_mean_relative_97.5P','LN_mean_relative_pvalue',
       'LN_mean_relative_pvalue_FDR']
    output_df = copy.deepcopy(input_df[temp_trait])
    return(output_df)

In [68]:
def Generate_Percentile_table(input_df):
    # generate a LN mean specific data table
    temp_q = [50,60,70,80,90,95,97,99]
    temp_list = []
    for x in temp_q:
        temp = str(x)+'_percentile_relative'
        temp_1 = temp +'_2.5P'
        temp_2 = temp +'_bootstrap_mean'
        temp_3 = temp +'_97.5P'
        temp_4 = temp + '_pvalue'
        temp_5 = temp + '_pvalue_FDR'
        temp_6 = temp + '_bootstrap_median'
        temp_list.append(temp)
        temp_list.append(temp_1)
        temp_list.append(temp_2)
        temp_list.append(temp_3)
        temp_list.append(temp_4)
        temp_list.append(temp_5)
        temp_list.append(temp_6)
    temp_trait = ['gRNA','Targeted_gene_name','Numbered_gene_name','Type'] + temp_list
    output_df = copy.deepcopy(input_df[temp_trait])
    return(output_df)

In [69]:
def Generate_Tumor_number_table(input_df):
    # generate a LN mean specific data table
    temp_trait = ['gRNA','Targeted_gene_name','Numbered_gene_name','Type','TTN_normalized_relative',
       'TTN_normalized_relative_2.5P', 'TTN_normalized_relative_bootstrap_mean','TTN_normalized_relative_bootstrap_median', 'TTN_normalized_relative_97.5P','TTN_normalized_relative_pvalue',
       'TTN_normalized_relative_pvalue_FDR']
    output_df = copy.deepcopy(input_df[temp_trait])
    return(output_df)

In [70]:
def Generate_Tumor_burden_table(input_df):
    # generate a LN mean specific data table
    temp_trait = ['gRNA','Targeted_gene_name','Numbered_gene_name','Type','TTB_normalized_relative',
       'TTB_normalized_relative_2.5P', 'TTB_normalized_relative_bootstrap_mean','TTB_normalized_relative_bootstrap_median', 'TTB_normalized_relative_97.5P','TTB_normalized_relative_pvalue',
       'TTB_normalized_relative_pvalue_FDR']
    output_df = copy.deepcopy(input_df[temp_trait])
    return(output_df)

In [71]:
def Generate_seperate_table(input_df,input_mouse_type,input_screening):
    df1 = Generate_LN_mean_table(input_df)
    df1['Mouse_genotype'] = input_mouse_type
    df1['Screening'] = input_screening
    df2 = Generate_Percentile_table(input_df)
    df2['Mouse_genotype'] = input_mouse_type
    df2['Screening'] = input_screening
    df3 = Generate_Tumor_number_table(input_df)
    df3['Mouse_genotype'] = input_mouse_type
    df3['Screening'] = input_screening
    df4 = Generate_Tumor_burden_table(input_df)
    df4['Mouse_genotype'] = input_mouse_type
    df4['Screening'] = input_screening
    return df1,df2,df3,df4

### 4.1 Initial screening

In [72]:
Initial_screening_LN_mean,Initial_screening_percentile,Initial_screening_TTN,Initial_screening_TTB = Generate_seperate_table(Chromatin_df_final,'KTC','Initial_screening')
Initial_screening_KT_LN_mean,Initial_screening_KT_percentile,Initial_screening_KT_TTN,Initial_screening_KT_TTB = Generate_seperate_table(Chromatin_KT_df_final,'KT','Initial_screening')

### 4.2 Saturation screening

In [73]:
Saturation_screening_LN_mean,Saturation_screening_percentile,Saturation_screening_TTN,Saturation_screening_TTB = Generate_seperate_table(Saturation_df_final,'KTC','Saturation_screening')
Saturation_screening_KT_LN_mean,Saturation_screening_KT_percentile,Saturation_screening_KT_TTN,Saturation_screening_KT_TTB = Generate_seperate_table(Saturation_KT_df_final,'KT','Saturation_screening')

### 4.3 Combine data

In [74]:
LN_mean_table = pd.concat([Initial_screening_LN_mean.reset_index(drop = True),
                           Initial_screening_KT_LN_mean.reset_index(drop = True),
                           Saturation_screening_LN_mean.reset_index(drop = True),
                           Saturation_screening_KT_LN_mean.reset_index(drop = True)])
LN_mean_table = LN_mean_table.sort_values(by = ['Screening','Mouse_genotype','Numbered_gene_name'],ascending = [True,False,True]).reset_index(drop = True)

In [75]:
Percentile_table = pd.concat([Initial_screening_percentile.reset_index(drop = True),
                           Initial_screening_KT_percentile.reset_index(drop = True),
                           Saturation_screening_percentile.reset_index(drop = True),
                           Saturation_screening_KT_percentile.reset_index(drop = True)])
Percentile_table = Percentile_table.sort_values(by = ['Screening','Mouse_genotype','Numbered_gene_name'],ascending = [True,False,True]).reset_index(drop = True)

In [76]:
TTN_table = pd.concat([Initial_screening_TTN.reset_index(drop = True),
                           Initial_screening_KT_TTN.reset_index(drop = True),
                           Saturation_screening_TTN.reset_index(drop = True),
                           Saturation_screening_KT_TTN.reset_index(drop = True)])
TTN_table = TTN_table.sort_values(by = ['Screening','Mouse_genotype','Numbered_gene_name'],ascending = [True,False,True]).reset_index(drop = True)

In [77]:
TTB_table = pd.concat([Initial_screening_TTB.reset_index(drop = True),
                           Initial_screening_KT_TTB.reset_index(drop = True),
                           Saturation_screening_TTB.reset_index(drop = True),
                           Saturation_screening_KT_TTB.reset_index(drop = True)])
TTB_table = TTB_table.sort_values(by = ['Screening','Mouse_genotype','Numbered_gene_name'],ascending = [True,False,True]).reset_index(drop = True)

### 4.4 Output_data

In [80]:
LN_mean_table.to_csv(LN_mean_output_address,index = False)
Percentile_table.to_csv(Percentile_output_address,index = False)
TTN_table.to_csv(TTN_output_address,index = False)
TTB_table.to_csv(TTB_output_address,index = False)

## 5 Gene level effect

In [81]:
def Cal_Combined_Effect(x,bootstrap_N):
    d = {}
    temp_trait_of_interest = ['LN_mean_relative_bootstrap_mean','TTB_normalized_relative_bootstrap_mean','95_percentile_relative_bootstrap_mean','TTN_normalized_relative_bootstrap_mean'] # this is the list that I am interested to study gene level effect
    temp_weight_list = x['TTN_normalized_relative']
    for temp_trait in temp_trait_of_interest:
        d[temp_trait.replace('_bootstrap_mean','_score')] = sum(x[temp_trait]*temp_weight_list/sum(temp_weight_list))
        d[temp_trait.replace('_bootstrap_mean','_pvalue')] = Stouffer_Test(x[temp_trait.replace('_bootstrap_mean','_pvalue')],temp_weight_list,bootstrap_N)
        d[temp_trait.replace('_bootstrap_mean','_pvalue_twoside')] = Stouffer_Test(x[temp_trait.replace('_bootstrap_mean','_pvalue_twoside')],temp_weight_list,bootstrap_N)
    return pd.Series(d, index=list(d.keys())) 

In [82]:
def Stouffer_Test(input_pvalue_list,input_weight,bootstrap_N):
    # this function combine pvalue using the Stouffer weighted Z 
    temp_corrected_p_list = []
    for tp in input_pvalue_list:
        if tp ==1:
            temp_corrected_p_list.append(1-1/bootstrap_N) #compensate for p=1
        elif tp == 0:
            temp_corrected_p_list.append(0+1/bootstrap_N)#compensate for p=0
        else:
            temp_corrected_p_list.append(tp)
    z_score = scipy.stats.norm.ppf(temp_corrected_p_list)# use Percent point function to calculate z score based on pvalue
    z_weighted = sum(np.array(input_weight)*z_score)/math.sqrt(sum(np.array(input_weight)**2)) # calculate weighted z_score
    stouffer_alpha  = scipy.stats.norm.cdf(z_weighted)
    return(stouffer_alpha)

In [83]:
def Generate_Gene_Level_Effect(input_df,bootstrap_N):
    temp_df = input_df.groupby(['Targeted_gene_name', 'Type'],as_index = False).apply(Cal_Combined_Effect,(bootstrap_N))
    temp_trait_of_interest = ['LN_mean_relative','TTB_normalized_relative','95_percentile_relative','TTN_normalized_relative']
    for temp_trait in temp_trait_of_interest:
        temp_name1 = temp_trait + '_pvalue'
        temp_name2 = temp_name1 + '_FDR'
        temp_name3 = temp_name1 + '_twoside'
        temp_name4 = temp_name1 + '_twoside_FDR'
        temp_df[temp_name2] = fdr(temp_df[temp_name1])
        temp_df[temp_name4] = fdr(temp_df[temp_name3])
    return(temp_df)

In [84]:
Chromatin_gene_level_df = Generate_Gene_Level_Effect(Chromatin_df_final,1000)

In [85]:
Chromatin_gene_level_KT_df = Generate_Gene_Level_Effect(Chromatin_KT_df_final,1000)

In [86]:
Saturation_gene_level_df = Generate_Gene_Level_Effect(Saturation_df_final,1000)

In [87]:
Saturation_gene_level_KT_df = Generate_Gene_Level_Effect(Saturation_KT_df_final,1000)

In [88]:
Chromatin_gene_level_df.sort_values(by = 'LN_mean_relative_pvalue')

,Targeted_gene_name,Type,LN_mean_relative_score,LN_mean_relative_pvalue,LN_mean_relative_pvalue_twoside,TTB_normalized_relative_score,TTB_normalized_relative_pvalue,TTB_normalized_relative_pvalue_twoside,95_percentile_relative_score,95_percentile_relative_pvalue,...,TTN_normalized_relative_pvalue,TTN_normalized_relative_pvalue_twoside,LN_mean_relative_pvalue_FDR,LN_mean_relative_pvalue_twoside_FDR,TTB_normalized_relative_pvalue_FDR,TTB_normalized_relative_pvalue_twoside_FDR,95_percentile_relative_pvalue_FDR,95_percentile_relative_pvalue_twoside_FDR,TTN_normalized_relative_pvalue_FDR,TTN_normalized_relative_pvalue_twoside_FDR
72,Ing1,Experiment,1.190761,2.345095e-09,8.359588e-09,1.339999,1.297310e-03,1.234433e-02,1.221909,8.930495e-06,...,1.751745e-05,1.227453e-04,3.325179e-07,4.095385e-07,3.312548e-03,3.151999e-02,4.605334e-05,0.000207,5.420106e-05,3.753722e-04
63,Gtf3c1,Experiment,0.804770,4.629252e-09,1.543318e-08,0.480059,5.084374e-07,1.041954e-06,0.759553,7.825053e-08,...,4.978015e-08,4.978015e-08,3.325179e-07,4.095385e-07,3.518922e-06,7.211418e-06,8.947778e-07,0.000006,3.216707e-07,3.457960e-07
36,Chd2,Experiment,0.843985,6.015470e-09,2.255046e-08,0.828292,1.088104e-02,1.986008e-01,0.805524,1.559251e-06,...,1.299436e-04,9.290100e-04,3.325179e-07,4.095385e-07,2.104200e-02,3.553198e-01,1.108332e-05,0.000073,3.164367e-04,2.283454e-03
206,Yeats4,Experiment,0.813327,6.548837e-09,1.282523e-08,0.446456,6.558968e-08,6.558968e-08,0.791315,1.099398e-05,...,6.558968e-08,6.558968e-08,3.325179e-07,4.095385e-07,5.948305e-07,7.187535e-07,5.455505e-05,0.000365,3.833352e-07,4.107163e-07
112,Mllt6,Experiment,1.174621,8.949453e-09,3.151943e-08,1.300431,1.292001e-03,9.382443e-03,1.209808,1.704532e-06,...,6.209363e-04,1.019487e-02,3.325179e-07,4.095385e-07,3.312548e-03,2.492507e-02,1.179716e-05,0.000045,1.298404e-03,2.111221e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,sgSafe28,Inert,1.002645,4.484000e-01,8.968000e-01,1.119001,1.161000e-01,2.322000e-01,1.015394,3.823000e-01,...,3.689000e-01,7.378000e-01,4.553251e-01,9.403101e-01,1.615571e-01,4.044278e-01,4.070644e-01,0.874303,3.912125e-01,8.257081e-01
257,sgSafe42,Inert,1.002274,4.533000e-01,9.066000e-01,1.001093,4.889000e-01,9.778000e-01,0.993153,4.299000e-01,...,3.632000e-01,7.264000e-01,4.585304e-01,9.403101e-01,4.889000e-01,9.778000e-01,4.468921e-01,0.922969,3.867271e-01,8.234621e-01
260,sgSafe7,Inert,1.002403,4.659000e-01,9.318000e-01,1.035160,3.827000e-01,7.654000e-01,0.994752,4.565000e-01,...,4.189000e-01,8.378000e-01,4.684813e-01,9.478154e-01,4.058472e-01,8.639494e-01,4.649718e-01,0.940841,4.334859e-01,8.914427e-01
242,sgSafe29,Inert,1.002051,4.667000e-01,9.334000e-01,0.970671,4.034000e-01,8.068000e-01,1.010981,4.412000e-01,...,2.100000e-02,4.200000e-02,4.684813e-01,9.478154e-01,4.210087e-01,8.841183e-01,4.532641e-01,0.928285,3.430435e-02,7.565753e-02


### 5.1 Output_data

In [89]:
temp_a = parent_address + "Chromatin/Chromatin_gene_result_final.csv"

In [90]:
Chromatin_gene_level_df.to_csv(temp_a,index = False)

In [91]:
temp_a = parent_address + "Chromatin/Chromatin_gene_result_KT_final.csv"

In [92]:
Chromatin_gene_level_KT_df.to_csv(temp_a,index = False)

In [93]:
temp_a = parent_address + "Saturation/Saturation_gene_result_final.csv"

In [94]:
Saturation_gene_level_df.to_csv(temp_a,index = False)

In [95]:
temp_a = parent_address + "Saturation/Saturation_gene_result_KT_final.csv"

In [96]:
Saturation_gene_level_KT_df.to_csv(temp_a,index = False)